# Demo to play around with langchain summarization


In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/huggingface/hub'
# os.environ['HUGGINGFACE_HUB_CACHE'] = '/mnt/huggingface/hub/'

import torch

import transformers
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

# For model
from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# For summarization
from langchain.schema.document import Document
from langchain.document_loaders import TextLoader
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter

/data/anaconda/envs/logzilla/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load model config

In [2]:
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name
)

Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Bitsandbytes for quantization config

In [4]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Load model

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

Loading checkpoint shards: 100%|██████████| 2/2 [02:23<00:00, 71.87s/it]


Using Langchain

In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    #temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
    #do_sample=True
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [9]:
llm("Jet fuels don't melt steel beams")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


".\n\nComment: @JamesK.Politano I think you are confusing the two. The OP is talking about a car that has been in an accident and is on fire. The question is whether or not it would be safe to get out of the car.\n\nComment: @JamesK.Politano I was thinking about the same thing, but I thought maybe the OP meant that the car was on fire because of the fuel leak.\n\n## Answer (1)\n\nI'm not sure if this is relevant, but I have seen cars on fire before. In one case, the car was parked in a parking lot and someone had left their cigarette lighter on. The car caught fire and burned down completely.\n\nIn another case, a car was involved in a high-speed crash and caught fire. The driver managed to escape, but the car was completely destroyed.\n\nIn both cases, the flames were intense enough to melt metal. However, they were also intense enough to cause severe burns to anyone who came near them.\n\nSo, if you were in a car that was on fire, it would probably be best to stay inside until the fi

Summarization test

In [13]:
# Load documents
loader = TextLoader("./logs/cleaned_test_log1.out")
docs = loader.load()

# Define prompt template and chains for mapping and reducing
map_template = """The following is a set of entries from a system log file:
{docs}
Based on this snippet of a system log file, please identify the main events that
has happened during the log. 

Answer in less than 3 sentences. 

Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """The following is set of summaries of events that happened in a log file:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main events that
happened within the log.

Answer in less than 3 sentences.

Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=True,
    verbose=True,
)

# Separator is important, it is by default `\n\n` (probably)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=10, separator="\n"
)
split_docs = text_splitter.split_documents(docs)



In [14]:
output = map_reduce_chain(split_docs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new MapReduceDocumentsChain chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


In [15]:
output["output_text"]

'\n\nThe main events that happened within the log include the system booting up, starting the kernel modules, mounting the root filesystem, starting the journal service, generating network units, starting a rule-based manager for device events and files, skipping several bind mounts due to failed condition checks, deactivating or stopping various services, starting loading kernel modules, mounting the temporary directory /tmp, finishing loading kernel modules and starting the graphical interface.'

## HuggingFace Summarizer

In [16]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/huggingface/hub'
# os.environ['HUGGINGFACE_HUB_CACHE'] = '/mnt/huggingface/hub/'

import torch

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

In [21]:
model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name) #used for tokenization
summarizer = pipeline(model=model_name)

In [18]:
text_to_summarize = """
Nov 09 13:11:35 localhost kernel: rcu:         RCU restricting CPUs from NR_CPUS=64 to nr_cpu_ids=16.
Nov 09 13:11:35 localhost kernel:         Trampoline variant of Tasks RCU enabled.
Nov 09 13:11:35 localhost kernel:         Rude variant of Tasks RCU enabled.
Nov 09 13:11:35 localhost kernel:         Tracing variant of Tasks RCU enabled.
Nov 09 13:11:35 localhost kernel: Console: colour dummy device 80x25
Nov 09 13:11:35 localhost kernel: printk: console [tty0] enabled
Nov 09 13:11:35 localhost kernel: audit: initializing netlink subsys (disabled)
Nov 09 13:11:35 localhost kernel: audit: type=2000 audit(1699535493.145:1): state=initialized audit_enabled=0 res=1
Nov 09 13:11:35 localhost kernel: ACPI: Added _OSI(Linux-HPI-Hybrid-Graphics)
Nov 09 13:11:35 localhost kernel: ACPI: 7 ACPI AML tables successfully acquired and loaded
Nov 09 13:11:35 localhost kernel: ACPI: [Firmware Bug]: BIOS _OSI(Linux) query ignored
Nov 09 13:11:35 localhost kernel: ACPI: Interpreter enabled
Nov 09 13:11:35 localhost kernel: pci 0000:08:00.0: BAR 7: no space for [mem size 0x00800000 64bit pref]
Nov 09 13:11:35 localhost kernel: pci 0000:08:00.0: BAR 7: failed to assign [mem size 0x00800000 64bit pref]
Nov 09 13:11:35 localhost kernel: pci 0000:08:00.0: BAR 10: no space for [mem size 0x00200000 64bit pref]
Nov 09 13:11:35 localhost kernel: pci 0000:08:00.0: BAR 10: failed to assign [mem size 0x00200000 64bit pref]
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.1: PME: Signaling with IRQ 25
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.1: AER: enabled with IRQ 25
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.3: PME: Signaling with IRQ 26
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.3: AER: enabled with IRQ 26
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.5: PME: Signaling with IRQ 27
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:01.5: AER: enabled with IRQ 27
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:08.2: PME: Signaling with IRQ 29
Nov 09 13:11:35 localhost kernel: pcieport 0000:00:08.2: AER: enabled with IRQ 29
Nov 09 13:11:35 localhost kernel: piix4_smbus 0000:00:14.0: Auxiliary SMBus Host Controller at 0xb20
Nov 09 13:11:35 localhost kernel: fail to initialize ptp_kvm
Nov 09 13:11:35 localhost kernel: microcode: Microcode Update Driver: v2.2.
Nov 09 13:11:35 localhost kernel: IPI shorthand broadcast: enabled
Nov 09 13:11:35 localhost kernel: Key type encrypted registered
Nov 09 13:11:35 localhost kernel: printk: console [netcon0] enabled
Nov 09 13:11:35 localhost kernel: sd 0:0:0:0: [sda] Mode Sense: 00 3a 00 00
Nov 09 13:11:35 localhost kernel: sd 0:0:0:0: [sda] Write cache: enabled, read cache: enabled, doesn't support DPO or FUA
Nov 09 13:11:35 localhost systemd[1]: Listening on udev Kernel Socket.
Nov 09 13:11:35 localhost systemd[1]: Huge Pages File System was skipped because of a failed condition check (ConditionPathExists=/sys/kernel/mm/hugepages).
Nov 09 13:11:35 localhost systemd[1]: Mounting /var/volatile...
Nov 09 13:11:35 localhost systemd[1]: Create List of Static Device Nodes was skipped because of a failed condition check (ConditionFileNotEmpty=/lib/modules/5.15.73/modules.devname).
Nov 09 13:11:35 localhost systemd[1]: Starting Mount sysroot partition...
Nov 09 13:11:35 localhost systemd[1]: Rebuild Hardware Database was skipped because of a failed condition check (ConditionNeedsUpdate=/etc).
Nov 09 13:11:35 localhost systemd[1]: Starting Journal Service...
Nov 09 13:11:35 localhost systemd[1]: Load Kernel Modules was skipped because all trigger condition checks failed.
Nov 09 13:11:35 localhost systemd[1]: Starting Generate network units from Kernel command line...
Nov 09 13:11:35 localhost systemd[1]: Platform Persistent Storage Archival was skipped because of a failed condition check (ConditionDirectoryNotEmpty=/sys/fs/pstore).
Nov 09 13:11:35 localhost systemd[1]: Mounted /var/volatile.
Nov 09 13:11:35 localhost systemd[1]: modprobe@configfs.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Finished Load Kernel Module configfs.
Nov 09 13:11:35 localhost systemd[1]: modprobe@fuse.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Reached target Preparation for Network.
Nov 09 13:11:35 localhost systemd[1]: FUSE Control File System was skipped because of a failed condition check (ConditionPathExists=/sys/fs/fuse/connections).
Nov 09 13:11:35 localhost kernel: audit: type=1334 audit(1699535495.186:2): prog-id=5 op=LOAD
Nov 09 13:11:35 localhost kernel: BTRFS info (device sda2): disk space caching is enabled
Nov 09 13:11:35 localhost systemd[1]: Starting Rule-based Manager for Device Events and Files...
Nov 09 13:11:35 localhost systemd[1]: Bind mount volatile /var/cache was skipped because of a failed condition check (ConditionPathIsReadWrite=!/var/cache).
Nov 09 13:11:35 localhost systemd[1]: Bind mount volatile /var/spool was skipped because of a failed condition check (ConditionPathIsReadWrite=!/var/spool).
Nov 09 13:11:35 localhost systemd[1]: Bind mount volatile /srv was skipped because of a failed condition check (ConditionPathIsReadWrite=!/srv).
Nov 09 13:11:35 localhost systemd[1]: Reached target Local File Systems.
Nov 09 13:11:35 localhost systemd[1]: Store a System Token in an EFI Variable was skipped because of a failed condition check (ConditionPathExists=/sys/firmware/efi/efivars/LoaderFeatures-4a67b082-0a4c-41cf-b6c7-440b29bb8c4f).
Nov 09 13:11:35 localhost systemd[1]: Starting Automatic Boot Loader Update...
Nov 09 13:11:35 localhost systemd[1]: Commit a transient machine-id on disk was skipped because of a failed condition check (ConditionPathIsMountPoint=/etc/machine-id).
Nov 09 13:11:35 localhost systemd[1]: Merge System Extension Images into /usr/ and /opt/ was skipped because all trigger condition checks failed.
Nov 09 13:11:35 localhost systemd-udevd[202]: /lib/udev/rules.d/50-udev-default.rules:44 Unknown group 'sgx', ignoring
Nov 09 13:11:35 localhost systemd[1]: Flush Journal to Persistent Storage was skipped because of a failed condition check (ConditionPathExists=!/etc/initrd-release).
Nov 09 13:11:35 localhost systemd[1]: Finished Create Volatile Files and Directories.
Nov 09 13:11:35 localhost systemd[1]: Rebuild Journal Catalog was skipped because of a failed condition check (ConditionNeedsUpdate=/var).
Nov 09 13:11:35 localhost systemd[1]: Update is Completed was skipped because all trigger condition checks failed.
Nov 09 13:11:35 localhost systemd[1]: Reached target System Initialization.
Nov 09 13:11:35 localhost systemd[1]: Daily Cleanup of Temporary Directories was skipped because of a failed condition check (ConditionPathExists=!/etc/initrd-release).
Nov 09 13:11:35 localhost systemd[1]: System is tainted: cgroupsv1
Nov 09 13:11:35 localhost systemd[1]: initrd-parse-etc.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped target Path Units.
Nov 09 13:11:35 localhost systemd[1]: systemd-ask-password-console.path: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped target Timer Units.
Nov 09 13:11:35 localhost systemd[1]: dbus.socket: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped target Swaps.
Nov 09 13:11:35 localhost systemd[1]: systemd-coredump.socket: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Closed Process Core Dump Socket.
Nov 09 13:11:35 localhost systemd[1]: systemd-boot-update.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped Automatic Boot Loader Update.
Nov 09 13:11:35 localhost systemd[1]: systemd-sysctl.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped Apply Kernel Variables.
Nov 09 13:11:35 localhost systemd[1]: systemd-udev-trigger.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopping Record System Boot/Shutdown in UTMP...
Nov 09 13:11:35 localhost systemd[1]: initrd-cleanup.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Finished Cleaning Up and Shutting Down Daemons.
Nov 09 13:11:35 localhost systemd[1]: systemd-update-utmp.service: Deactivated successfully.
Nov 09 13:11:35 localhost systemd[1]: Stopped Record System Boot/Shutdown in UTMP.
Nov 09 13:11:35 localhost systemd[1]: systemd-tmpfiles-setup.service: Deactivated successfully.
Nov 09 13:11:35 localhost kernel: hub 3-2.4:1.0: 7 ports detected
Nov 09 13:11:36 localhost systemd[1]: systemd-udevd.service: Deactivated successfully.
Nov 09 13:11:36 localhost systemd[1]: Stopped Rule-based Manager for Device Events and Files.
Nov 09 13:11:36 localhost systemd[1]: systemd-udevd-control.socket: Deactivated successfully.
Nov 09 13:11:36 localhost systemd[1]: Closed udev Control Socket.
Nov 09 13:11:36 localhost systemd[1]: systemd-udevd-kernel.socket: Deactivated successfully.
Nov 09 13:11:36 localhost systemd[1]: Starting Cleanup udev Database...
Nov 09 13:11:36 localhost systemd[1]: systemd-network-generator.service: Deactivated successfully.
Nov 09 13:11:36 localhost systemd[1]: Stopped Generate network units from Kernel command line.
Nov 09 13:11:36 localhost systemd[1]: systemd-tmpfiles-setup-dev.service: Deactivated successfully.
Nov 09 13:11:36 localhost systemd[1]: Stopped Create Static Device Nodes in /dev.
Nov 09 13:11:36 localhost systemd[1]: initrd-udevadm-cleanup-db.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 kernel: usb 3-2.4.1: new high-speed USB device number 7 using xhci_hcd
Nov 09 13:11:36 CMX50070-101776 systemd[1]: initrd-switch-root.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Listening on udev Kernel Socket.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Huge Pages File System was skipped because of a failed condition check (ConditionPathExists=/sys/kernel/mm/hugepages).
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Mounting Temporary Directory /tmp...
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Create List of Static Device Nodes was skipped because of a failed condition check (ConditionFileNotEmpty=/lib/modules/5.15.73/modules.devname).
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Starting Load Kernel Module tun...
Nov 09 13:11:36 CMX50070-101776 systemd[1]: mount-sysroot.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Starting RPC Bind...
Nov 09 13:11:36 CMX50070-101776 systemd[1]: systemd-fsck-root.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 audit: BPF prog-id=0 op=UNLOAD
Nov 09 13:11:36 CMX50070-101776 audit: CONFIG_CHANGE op=set audit_enabled=1 old=1 auid=4294967295 ses=4294967295 res=1
Nov 09 13:11:36 CMX50070-101776 audit[281]: SYSCALL arch=c000003e syscall=46 success=yes exit=60 a0=3 a1=7fffbc10fe50 a2=4000 a3=7fffbc10feec items=0 ppid=1 pid=281 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm="systemd-journal" exe="/lib/systemd/systemd-journald" key=(null)
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Queued start job for default target Graphical Interface.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: systemd-journald.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Mounted Temporary Directory /tmp.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: modprobe@configfs.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Finished Load Kernel Module configfs.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: modprobe@dm-mod.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Finished Load Kernel Module dm-mod.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: modprobe@drm.service: Deactivated successfully.
Nov 09 13:11:36 CMX50070-101776 systemd[1]: Finished Load Kernel Module drm.
"""

In [20]:
#summarizer(text_to_summarize)

Chunking

In [22]:
import numpy as np
from scipy.spatial.distance import cosine
import torch

def split_context(context, overlap=50, max_length=1024):
    """
    Function for splitting context into overlapping chunks.
    
    param context: This is the text that you want to split into chunks. 
    The function will split this text based on the max_length and overlap parameters.

    param overlap (default=50): This is the number of characters that will overlap between each chunk. 
    This is used to ensure that the context is not cut off in the middle of a sentence, which could make the text difficult to understand.

    param max_length (default=512): This is the maximum length of each chunk. 
    The function will split the context into chunks of this length, with the exception of the last chunk, which may be shorter.

    The function returns a list of chunks, where each chunk is a string of text from the context. 
    The chunks are created by starting at the beginning of the context and moving forward max_length
    characters at a time, with an overlap of overlap characters between each chunk.
    """
    
    chunks = []
    start = 0
    while start < len(context):
        end = min(start + max_length, len(context))
        chunks.append(context[start:end])
        if end == len(context):
            break
        start = end - overlap
    return chunks

In [23]:
chunks = split_context(text_to_summarize)

for chunk in chunks:
    summary = summarizer(chunk)
    print(summary)

[{'summary_text': ' Nov 09 13:11:35 localhost kernel: rcu:    RCU restricting CPUs from NR_CPUS=64 to nr_cpu_ids=16 . Nov 09:11 \xa0localhost\xa0kernels: Console: colour dummy device 80x25. Nov 09 \xa0\xa0\xa0- Nov 09\xa013:11\xa014:30:\xa0Console: console [tty0] enabled .'}]
[{'summary_text': ' Preter enabled: No space for [mem size 0x00800000 64bit pref] preter enabled . Preter is enabled with IRQ 25. preter enable . No space is assigned to [mem] size \xa00x00200000 64bit preter preter .'}]
[{'summary_text': ' l: pcieport 0000:00:08.2: PME: Signaling with IRQ 29: AER: enabled . v2.2.0: Auxiliary SMBus Host Controller at 0x20: 0x4_smbus . The failed condition check on Huge Pages File System was skipped .'}]
[{'summary_text': ' Nov 09 13:11:35 localhost systemd[1]: Mounting /var/volatile . Nov 09 :13:35:05:01:02:02 :02:03:05 :08:06:08:08 :06:09:05 . Nov09 13:10:06 :05:07:08 .'}]
[{'summary_text': ' FUSE Control File System was skipped because of a failed condition check (ConditionPathE

Your max_length is set to 142, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


[{'summary_text': ' t=60 a0=3 a1=7fffbc10fe50 a2=4000 a3=7ffBC10feec items=0 ppid=1 pid=281 auid=4294967295 uid=0 gid =0 gids=0 euid=0 suid= 0 egid=\u2009+++:\xa0Systemd-journald.service: Deactivated successfully . The default target for the Graphical Interface is Graphical Interfaces .'}]
[{'summary_text': ' Drm. :36 CMX5007070-101776: Finished Load Kernel Module drm. drm . :36: Finished Loaded Kernel Module . Drm: Finished loaded kernel module . CMX 5007070: Finished loading kernel module. CMX550070: Loaded kernel module for the kernel .'}]
